In [17]:
import psycopg2

def list_tables_and_columns(conn):
    """Fetches and prints all tables and their associated columns in the database."""
    cur = conn.cursor()

    # ✅ Step 1: Get all table names
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)
    tables = cur.fetchall()

    if not tables:
        print("❌ No tables found in the database.")
        return

    print("✅ Tables and their columns:\n")

    # ✅ Step 2: Iterate through tables and fetch column details
    for table in tables:
        table_name = table[0]
        print(f"📌 Table: {table_name}")
        # cur.execute(f"""DROP TABLE IF EXISTS {table_name};""")
        cur.execute(f"""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = '{table_name}'
            ORDER BY ordinal_position;
        """)
        columns = cur.fetchall()

        for column in columns:
            print(f"   ➜ Column: {column[0]} ({column[1]})")
        print("-" * 50)

    cur.close()
db_host = 'database-1.c3ww44kcayjx.ap-south-1.rds.amazonaws.com'
db_name = 'database-1'
db_user = 'postgres'
db_password = 'datascience'
# ✅ Usage Example
conn = psycopg2.connect(
    host=db_host,
    user=db_user,
    password=db_password
)

list_tables_and_columns(conn)

# ✅ Close connection
# conn.close()


✅ Tables and their columns:

📌 Table: total_revenue_per_branch
   ➜ Column: branch (text)
   ➜ Column: total_revenue (double precision)
--------------------------------------------------


In [18]:
import psycopg2

def show_all_rows(conn):
    """Fetches and prints all rows from every table in the database."""
    cur = conn.cursor()

    # ✅ Step 1: Get all table names
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)
    tables = cur.fetchall()

    if not tables:
        print("❌ No tables found in the database.")
        return

    print("✅ Fetching all rows from each table:\n")

    # ✅ Step 2: Iterate through tables and fetch all rows
    for table in tables:
        table_name = table[0]
        print(f"📌 Table: {table_name}")

        cur.execute(f"SELECT * FROM {table_name} LIMIT 10;")  # Fetch first 10 rows (adjust as needed)
        rows = cur.fetchall()

        if not rows:
            print("   ❌ No data found in this table.\n")
        else:
            # ✅ Step 3: Print column names
            column_names = [desc[0] for desc in cur.description]
            print(f"   ➜ Columns: {column_names}")

            # ✅ Step 4: Print row data
            for row in rows:
                print(f"   ➜ {row}")

        print("-" * 50)

    cur.close()

# # ✅ Usage Example
# conn = psycopg2.connect(
#     host="your-db-host",
#     database="your-db-name",
#     user="your-username",
#     password="your-password"
# )

show_all_rows(conn)

# ✅ Close connection
conn.close()


✅ Fetching all rows from each table:

📌 Table: total_revenue_per_branch
   ➜ Columns: ['branch', 'total_revenue']
   ➜ ('C', 110568.70649999994)
   ➜ ('A', 106200.3705000001)
   ➜ ('B', 106197.67199999996)
--------------------------------------------------


In [14]:
import psycopg2

def drop_all_tables(conn):
    """Drops all tables in the public schema of the PostgreSQL database."""
    cur = conn.cursor()

    # ✅ Step 1: Get all table names
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    tables = cur.fetchall()

    if not tables:
        print("❌ No tables found in the database.")
        return

    print("🚨 Dropping all tables...")

    # ✅ Step 2: Disable foreign key constraints
    cur.execute("SET session_replication_role = 'replica';")  # Temporarily disable foreign key constraints

    # ✅ Step 3: Drop each table
    for table in tables:
        table_name = table[0]
        cur.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE;")
        print(f"✅ Dropped table: {table_name}")

    # ✅ Step 4: Re-enable foreign key constraints
    cur.execute("SET session_replication_role = 'origin';")

    # ✅ Step 5: Commit changes
    conn.commit()
    cur.close()
    print("🔥 All tables removed successfully!")

# ✅ Usage Example
# conn = psycopg2.connect(
#     host="your-db-host",
#     database="your-db-name",
#     user="your-username",
#     password="your-password"
# )

drop_all_tables(conn)

# ✅ Close connection
conn.close()


🚨 Dropping all tables...
✅ Dropped table: total_revenue_per_branch
🔥 All tables removed successfully!


In [19]:
import boto3

glue_client = boto3.client("glue")

while True:
    response = glue_client.get_job_runs(JobName="extract")

EntityNotFoundException: An error occurred (EntityNotFoundException) when calling the GetJobRuns operation: Job not found.

In [20]:
import yaml

In [22]:
yaml.safe_load(open("extract.yaml","r"))

{'source_bucket': 'etl-input-9062',
 'destination_bucket': 'etl-intermediate-9062',
 'source_key': 'supermarket_sales - Sheet1.csv',
 'destination_key': 'test.csv'}